In [0]:
%run ./01_Project_Config

Gold Dimensions

In [0]:
from pyspark.sql.functions import *

In [0]:
def process_gold_dimension(source_table, target_table, pk_column, sk_column, selected_columns):
    print(f"Processing {target_table}...")
    
    df_source = spark.table(f"{catalog}.silver.{source_table}")
    
    # 2. Generate Positive Numeric Surrogate Key (SK)
    df_gold = df_source.withColumn(sk_column, abs(xxhash64(col(pk_column).cast("string")))) \
                       .select(sk_column, *selected_columns) \
                       .withColumn("gold_processed_ts", current_timestamp())
    
    # 3. Write to Gold (Overwrite - SCD Type 1)
    (df_gold.write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(f"{catalog}.gold.{target_table}"))
    
    # 4. OPTIMIZE & Z-ORDER (Performance Booster)
    print(f":) Optimizing {target_table} with Z-Order on {sk_column}...")
    spark.sql(f"OPTIMIZE {catalog}.gold.{target_table} ZORDER BY ({sk_column})")
    
    print(f" => {target_table} created & optimized.")
    display(df_gold.limit(3))

In [0]:
# 1. Product Dimension
cols_product = [
    "product_id", "product_name", "category", "brand", 
    "cost_price", "list_price", "launch_date"
]
process_gold_dimension(
    source_table="dim_product",
    target_table="dim_product_gold",
    pk_column="product_id",
    sk_column="product_key", 
    selected_columns=cols_product
)

In [0]:
# 2. Store Dimension
cols_store = [
    "store_id", "store_name", "region", "store_type", "city"
]
process_gold_dimension(
    source_table="dim_store",
    target_table="dim_store_gold",
    pk_column="store_id",
    sk_column="store_key",   
    selected_columns=cols_store
)

In [0]:
# 3. Customer Dimension
cols_customer = [
    "customer_id", "name", "email", "phone", 
    "age_band", "loyalty_tier", "city"
]
process_gold_dimension(
    source_table="dim_customer",
    target_table="dim_customer_gold",
    pk_column="customer_id",
    sk_column="customer_key", 
    selected_columns=cols_customer
)

In [0]:
# 4. Promotion Dimension
cols_promotion = [
    "promo_code", "promo_type", "discount_rule", "start_date", "end_date"
]
process_gold_dimension(
    source_table="dim_promotion",
    target_table="dim_promotion_gold",
    pk_column="promo_code",
    sk_column="promo_key",    
    selected_columns=cols_promotion
)
print("\nAll Gold Dimensions created successfully with Positive keys!")

Gold Sales Fact Table

In [0]:
df_sales = spark.read.table(f"{catalog}.silver.pos_sales")
df_returns = spark.read.table(f"{catalog}.silver.returns")

In [0]:
# 2. Prepare Returns (Aggregate return_qty by txn_id + product_id)
df_returns_agg = (df_returns
    .groupBy("txn_id", "product_id")
    .agg(sum("return_qty").alias("total_return_qty"),
        sum(col("return_qty") * col("return_qty")).alias("return_amount_check"))  
)

In [0]:
# 3. Join Sales + Returns (LEFT JOIN because not all sales have returns)
df_merged = df_sales.join(
    df_returns_agg, 
    on=["txn_id", "product_id"], 
    how="left"
)

In [0]:
# 4. Handle Nulls & Calculate Net Metrics
df_calculated = df_merged.withColumn("return_qty", coalesce(col("total_return_qty"), lit(0))) \
    .withColumn("net_quantity", col("quantity") - col("return_qty")) \
    .withColumn("return_amount", col("return_qty") * col("unit_price")) \
    .withColumn("net_sales", col("gross_sales") - col("discount_amount") - col("return_amount"))

In [0]:
# 5. SURROGATE KEY LOOKUP (Bringing in the Gold Keys)
dim_product = spark.read.table(f"{catalog}.gold.dim_product_gold")
dim_store = spark.read.table(f"{catalog}.gold.dim_store_gold")
dim_customer = spark.read.table(f"{catalog}.gold.dim_customer_gold")
dim_promo = spark.read.table(f"{catalog}.gold.dim_promotion_gold")

In [0]:
df_final_sales = (df_calculated
    .join(dim_product, "product_id")    # Get product_key
    .join(dim_store, "store_id")        # Get store_key
    .join(dim_customer, "customer_id")  # Get customer_key
    .join(dim_promo, "promo_code", "left") # Get promo_key (Left join kyuki promo null ho sakta hai)
    .select(
        # Keys
        col("txn_id"),
        col("product_key"),
        col("store_key"),
        col("customer_key"),
        coalesce(col("promo_key"), lit(-1)).alias("promo_key"),
        
        # Attributes
        col("txn_ts"),
        col("channel"),
        col("payment_type"),
        
        # Metrics
        col("quantity"),
        col("return_qty"),
        col("net_quantity"),
        col("unit_price"),
        col("gross_sales"),
        col("discount_amount"),
        col("return_amount"),
        col("net_sales"),
        
        # Audit
        current_timestamp().alias("gold_processed_ts")
    )
)

In [0]:
# 6. Write fact_sales
(df_final_sales.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{catalog}.gold.fact_sales"))

In [0]:
# Optimization: Z-Order by Keys for fast Dashboard filtering
spark.sql(f"OPTIMIZE {catalog}.gold.fact_sales ZORDER BY (store_key, product_key, txn_ts)")

print("fact_sales created & optimized.")

In [0]:
%sql
select * from retail_lakehouse.gold.fact_sales;

Gold Inventory Table


In [0]:
df_inventory = spark.read.table(f"{catalog}.silver.inventory")
dim_product = spark.read.table(f"{catalog}.gold.dim_product_gold") 
dim_store = spark.read.table(f"{catalog}.gold.dim_store_gold")

In [0]:
# Lookup SKs for Inventory
df_final_inventory = (df_inventory
    .join(dim_product, "product_id")
    .join(dim_store, "store_id")
    .select(
        col("snapshot_date"),
        col("store_key"),
        col("product_key"),
        coalesce(col("stock_on_hand"),lit(0)).alias("stock_on_hand"),
        col("reorder_point"),
        col("shrinkage_qty"),
        col("stockout_flag"),
        current_timestamp().alias("gold_processed_ts")
    )
)

In [0]:
# Write fact_inventory_daily
(df_final_inventory.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{catalog}.gold.fact_inventory_daily"))

In [0]:
# Optimization: Often filtered by Date and Store
spark.sql(f"OPTIMIZE {catalog}.gold.fact_inventory_daily ZORDER BY (snapshot_date, store_key)")

print("fact_inventory_daily created & optimized.")

In [0]:
%sql
select * from retail_lakehouse.gold.fact_inventory_daily;